# Benchmarking an algorithm used to solve a MAX-k-SAT problem

In this notebook we showcase functionality of qubrabench by implementing and benchmarking a hillclimbing algorithm to solve MAX-k-SAT, as described in https://arxiv.org/abs/2203.04975.
The paper describes two distinct variants of implementing the hillclimber: a simple hillclimber - which uses (quantum) search, and a steep one - which uses (quantum) max finding.

## Problem Definition: MAX-k-SAT

Max-k-SAT is a combinatorial optimization problem that given a list of clauses $(C_{i})^{p}_{i=1}$, each a disjunction of at most $k$ literals, and a set of weights $(w_{i})^{p}_{i=1}$, asks us to maximize the weight of the satisfied clauses,
$$\varphi(y) := \sum ^{p} _{i=1} w_{i}C_{i}(y),$$
over all assignments $y \in \{0, 1\}^{q}$ of the variables. This problem is NP-hard for $k ≥ 2$.

## Algorithm Definition: Hillclimber

For a given MAX-k-SAT problem instace, we start with a random variable assignment $y \in \{0, 1\}^{n}$ and look for improvements (higher values) of $\varphi$ in the set of all bitstrings that differ from $y$ in at most $d$ bits. We will only use $d=1$ for now, so this can be assumed to be the value of $d$ from this point on. All bitstrings that differ from our current solution $y$ in at most $d$ bits are collectively called the neighbourhood $N_{d}(y)$ of $y$.

### Simple Hillclimber
The simple hill climber randomly samples such assignments until it finds one with a strictly higher value of $\varphi$, which is then taken as the new assignment.
This procedure is repeated until no further improvement is found.
We can formalize each hillclimb step (described above) as searching for a solution in
$$f:N_{d}(y) \subseteq \{0,1\}^{n} →\{0,1\}$$
$$f(z) = \begin{cases}1 ~~~~~\text{if}~ \varphi(z)>\varphi(y)\\ 0 ~~~~~\text{otherwise} \end{cases}$$

### Steep Hillclimber

The steep hillclimber calculates the value of $\varphi$ for every neighbour $y^* \in N_{d}(y)$ of $y$. It then finds the neigbour with the highest value of $\varphi$ and uses this as the new assignment:
$$y_{new} = max_{y^* \in N_{d}(y)}(\varphi(y^*))$$
This procedure is repeated until no further improvement is found. 

# Solving a given problem instance

As mentioned above, we can use a hillclimber algorithm to solve the MAX-k-SAT problem. For the purpose of simplicity within this notebook, we will be focussing on the steep hillclimber and a naive implementation of the hillclimber algorithm. This repository also contains a hillclimber algorithm, which uses a more advanced approach and has much better performance than the naive approach. However, as we are focussing on the functionality of qubrabench in this notebook, this more complex implementation will not be covered in more detail.

## Implementing a naive steep hillclimber algorithm

To solve a MAX-k-SAT problem instance, consisting of a set auf clauses $(C_{i})^{p}_{i=1}$ and a set of weights $(w_{i})^{p}_{i=1}$, we will implement a naive steep hillclimber algorithm. We start by importing two libraries we will need in the course of this implementation.

In [ ]:
import itertools
import random

Before we can solve any problem instances, we must first create the problem instances. Let's assume that $k=3$, which means we will be solving a MAX-3-SAT problem. We will also assume that we have four variables available.

In [ ]:
d = 3
var_count = 4
demo_clauses_array = [[-1, -2, 3], [1, 2, -4], [-1, -2, 4]]
demo_weights_array = [3, 5, 1]

In this representation, each array in the demo_clauses_array represents one of the problems clauses. Each value in one of these clauses refers to one of the four available variables. If the value is negative, it represents a negated variable. The values in demo_weights_array represent the weight of the clause at the same index in the demo_clauses_array.

Now that we have an array of clauses and an array of weights representing our problem instance, we can start implementing the solver for this problem.

For this we implement a function, which randomly generates a sequence of zeros and ones. 

In [ ]:
def generate_random_solution(variable_count):
    return [random.randint(0, 1) for _ in range(variable_count)]


random_assignment = generate_random_solution(var_count)
print(random_assignment)

[0, 0, 1, 1]


Such a sequence represents a random variable assignment $y$ for our problem instance. We then start out from this variable assignment and take a look at the socalled neighbours. These are other variable assignments, which differ from our current assignment in exactly one place. For example the variable assignment (0, 0, 0) has the neighbours (1, 0, 0), (0, 1, 0) and (0, 0, 1). 

In [ ]:
def generate_differing_arrays(array, num_changes):
    index_combinations = itertools.combinations(range(len(array)), num_changes)
    arrays = set()

    for indices in index_combinations:
        for change_values in itertools.product([0, 1], repeat=num_changes):
            new_array = list(array)
            for i, j in zip(indices, change_values):
                new_array[i] = j
            arrays.add(tuple(new_array))

    arrays.discard(tuple(array))
    return list(arrays)


def get_neighbours(solution, max_hamming_distance):
    neighbours = set()
    for i in range(1, max_hamming_distance + 1):
        neighbours.update(generate_differing_arrays(solution, i))
    return neighbours


print(random_assignment)
demo_neighbours = get_neighbours(random_assignment, 1)
print(demo_neighbours)

[0, 0, 1, 1]
{(0, 0, 0, 1), (0, 0, 1, 0), (0, 1, 1, 1), (1, 0, 1, 1)}


For each of these neighbours we need to calculate the weight the clause has with this solution.

In [ ]:
def calculate_weight_for_solution(solution, clauses_array, weights_array):
    weight = 0
    for i in range(len(clauses_array)):
        for value in clauses_array[i]:
            if value > 0:
                if solution[value - 1] == 1:
                    weight += weights_array[i]
                    break
            elif value < 0:
                if solution[abs(value) - 1] == 0:
                    weight += weights_array[i]
                    break
    return solution, weight


print("Current weight:")
current_solution, current_weight = calculate_weight_for_solution(
    random_assignment, demo_clauses_array, demo_weights_array
)
print(random_assignment, current_weight)

Current weight:
[0, 0, 1, 1] 4


In [ ]:
def find_better_neighbour(solution, weight, clauses_array, weights_array, distance):
    highest_weight = -1
    highest_solution = []

    for neighbour in demo_neighbours:
        solution, weight = calculate_weight_for_solution(
            neighbour, demo_clauses_array, demo_weights_array
        )
        if weight > highest_weight:
            highest_weight = weight
            highest_solution = neighbour
        print(solution, weight)
    return highest_solution, highest_weight


print("Neighbour weights:")
highest_solution, highest_weight = find_better_neighbour(
    current_solution, current_weight, demo_clauses_array, demo_weights_array, d
)

print("Highest weight: " + str(highest_weight))
print("Highest solution: " + str(highest_solution))

Neighbour weights:
(0, 0, 0, 1) 4
(0, 0, 1, 0) 9
(0, 1, 1, 1) 9
(1, 0, 1, 1) 9
Highest weight: 9
Highest solution: (0, 0, 1, 0)


We then pick the neighbour with the highest weight, and if this weight is bigger than the current weight, we will use this solution and repeat this process until there is no more improvement. The optimal weight for a given assignment in this case is 9.

These steps we went through now form our hillclimber algorihm, which runs through this cycle multiple times.

In [ ]:
def hill_climber_sat(clauses_array, weights_array, variable_count, distance):
    # Example assignment, chosen to be able to demonstrate multiple steps
    current_solution = [0, 0, 0, 1]
    print("Starting with assignment: " + str(current_solution))

    solution, weight = calculate_weight_for_solution(
        current_solution, clauses_array, weights_array
    )
    print("Starting weight: " + str(weight))
    better_solution, better_weight = find_better_neighbour(
        solution, weight, clauses_array, weights_array, distance
    )

    while weight < better_weight:
        current_solution = better_solution
        weight = better_weight
        print("Found better weight: " + str(weight))
        better_solution, better_weight = find_better_neighbour(
            current_solution, weight, clauses_array, weights_array, distance
        )
    print("No better weight found.")
    print(
        "Completed. Optimal assignment: "
        + str(better_solution)
        + ". Weight: "
        + str(better_weight)
        + "."
    )


hill_climber_sat(demo_clauses_array, demo_weights_array, 4, 1)

Starting with assignment: [0, 0, 0, 1]
Starting weight: 4
(0, 0, 0, 1) 4
(0, 0, 1, 0) 9
(0, 1, 1, 1) 9
(1, 0, 1, 1) 9
Found better weight: 9
(0, 0, 0, 1) 4
(0, 0, 1, 0) 9
(0, 1, 1, 1) 9
(1, 0, 1, 1) 9
No better weight found.
Completed. Optimal assignment: (0, 0, 1, 0). Weight: 9.



## Spottig a pattern

If we take another look at this approach, we will notice, that we calculate the maximum value of a set of numbers within the find_better_neighbour function. In this case the maximum weight in a set of weights of all neighbours. This library offers a function, which enables a user to do exactly this and to keep track of runtime stats and approximations for call counts, if the max function was run on a quantum machine.

## Max Function

The library includes a set of common functions you would need in various circumstances, for example the search and the max function. These return the result you would expect from other implementations of these functions, and at the same time add in quantum benchmarking functionality. This simultaneously makes it easier to understand the use-cases of the library and makes it relatively easy to adapt existing code to use quantum benchmarking.

When we talk about quantum benchmarking here, we mean approximating the amount of calls needed to execute the function with identical parameters on a quantum computer. This library achieves this by counting the calls run on the classical (in this case most likely your) machine and basing performance assumptions on these measurements. All provided functions in qubrabench currently require a stats object, in which these collected statistics and approximations are stored.

Let's take a look at the docstrings of the function we will need to solve our problem - max.

In [ ]:
from qubrabench.algorithms.max import max as qmax

%pdoc qmax

Class docstring:
    Find the largest element in a list, while keeping track of query statistics.
    
    Args:
        iterable: iterable to find the maximum in
        default: default value to return if iterable is empty.
        key: function that maps iterable elements to values that are comparable. By default, use the iterable elements.
        error: upper bound on the failure probability of the quantum algorithm.
        stats: object that keeps track of statistics.
    
    Raises:
        ValueError: Raised when the failure rate `error` is not provided and statistics cannot be calculated.
        ValueError: Raised when iterable is an empty sequence and no default is provided.
    
    Returns:
        the desired maximum element
Call docstring:
    Call self as a function.

The following example will use the max function documented above. It may seem a bit confusing, that the values in the array (iterable) should be themselves arrays containing only a single integer, however this will be used here to demonstrate the use of the key function. We provide the max function with the described array and intend to receive the array containing the highest integer. The default value is set to [-1] here, such a value would indicate an empty iterable being passed to the function.

In [ ]:
from qubrabench.algorithms.max import max

max([[14], [2], [30], [7], [91]], default=[-1], key=lambda x: x[0])

[91]

Now that we have a basic understanding of how to use the max function, we can use it in our hillclimber algorithm. This will enable us to collect benchmarking results to approximate classical and quantum query counts for our problem instances. In the following section we will be running the hillclimber provided in this library, because it runs much faster for larger problem instances. This will nonetheless be sufficient to demonstrate how these benchmarking results may look.

## Benchmarking

The imported `run` function executes the hillclimber on a random instance and returns the statistics in a pandas DataFrame. As running the hillclimber also touches on some important topics on relation to the max function, we will also take a look at its code.

In [ ]:
from hillclimber import run

%psource run

def run(
    k: int,
    r: int,
    n: int,
    *,
    n_runs: int,
    rng: np.random.Generator,
    error: Optional[float] = None,
    random_weights: Optional[Callable[[int], npt.NDArray[W]]] = None,
    steep: bool = False,
) -> pd.DataFrame:
    """External interface to generate weighted sat instances, run the hillclimber algorithm and return statistics.

    Args:
        k: Number of literals in a clause
        r: Factor for the number of clauses
        n: size (variable number) of the SAT instances
        n_runs: number of runs to perform in each group
        rng: Source of randomness
        error: Upper bound on the failure rate. Defaults to None.
        random_weights: Optionally providable weights for SAT instance generation. Defaults to None.
        steep: Whether to perform hillclimb steep (greedily). Defaults to False.

    Returns:
        Dataframe holding benchmarking statistics of the runs performed.
    """
    assert n_runs >= 1

    history = []
    for run

As you can see, we can run the hillclimber multiple times in this case and then compute the average of the results. In the loop we first generate a QueryStats object, as mentioned above, which is used to store the results of the hillclimber run. After that, we create a problem instance using our desired parameters. Once this is done we can run the hillclimber, providing all possible parameters, including `error`, `stats` and `steep`. In our case we require the stats in a pandas dataframe, which is why we convert the stats object into a dictionary, add some additional interesting stats and add it to the `history` array.

Let's run the steep hill climber for $n = 100$, $n = 300$ and $n=1000$. We will run the hill climber five times for each $n$ and consistently use $k = 3$.

In [ ]:
import numpy as np

%%time
data_100 = run(
    k=3,
    r=3,
    n=100,
    n_runs=5,
    rng=np.random.default_rng(seed=100),
    error=10**-5,
    steep=True,
)
data_100

UsageError: Line magic function `%%time` not found.


In [ ]:
%%time
data_300 = run(
    k=3,
    r=3,
    n=300,
    n_runs=5,
    rng=np.random.default_rng(seed=100),
    error=10**-5,
    steep=True,
)
data_300

NameError: name 'np' is not defined

In [ ]:
%%time
data_500 = run(
    k=3,
    r=3,
    n=500,
    n_runs=5,
    rng=np.random.default_rng(seed=100),
    error=10**-5,
    steep=True,
)
data_500

NameError: name 'np' is not defined

## Plotting
Finally, after we have benchmarked the solving of a couple of our problem instances, we can take a look at the plotting functionality qubrabench provides. We use the `PlottingStrategy` wrapper to define our plot parameters and configuration.

In [ ]:
from qubrabench.utils.plotting_strategy import PlottingStrategy


class Plotter(PlottingStrategy):
    def __init__(self):
        self.colors[""] = "blue"

    def get_plot_group_column_names(self):
        return ["k", "r"]

    def get_data_group_column_names(self):
        """
        Generate a data line for each unique value in the specified columns.
        Useful if you the data was generated with different tags based on implementation source, parameter choice etc., that one wants to compare against in a single plot.

        Example: ["impl"] - a line will be generated for each unique `impl` label.
        """
        return []

    def compute_aggregates(self, data, *, quantum_factor):
        # compute combined query costs of quantum search
        c = data["quantum_expected_classical_queries"]
        q = data["quantum_expected_quantum_queries"]
        data["quantum_cost"] = c + quantum_factor * q
        return data

    def x_axis_column(self):
        return "n"

    def x_axis_label(self):
        return "$n$"

    def y_axis_label(self):
        return "Queries"

    def get_column_names_to_plot(self):
        return {
            "classical_actual_queries": ("Classical Queries", "o"),
            "quantum_cost": ("Quantum Queries", "x"),
        }

We can put all the benchmark stats in a single table and run the plotter. 

In [ ]:
%pdoc Plotter.plot

Class docstring:
    Plot benchmarking data.
    
    Args:
        data: a pandas DataFrame containing all the benchmark data.
        quantum_factor: conversion factor for the cost of a quantum query (w.r.t. classical queries).
        y_lower_lim: lower limit on the Y-axis (useful if the data starts at a large value)
    
    Raises:
        ValueError: if no columns are given to plot
Call docstring:
    Call self as a function.

In [ ]:
data = pd.concat([data_100, data_300, data_500])
Plotter().plot(data, quantum_factor=2, y_lower_lim=10)

NameError: name 'pd' is not defined

Now we can also run the "steep" hillclimber for the above instance sizes, and compare the two benchmarks.

In [ ]:
%%time
data_steep = [
    run(
        k=3,
        r=3,
        n=n,
        n_runs=5,
        rng=np.random.default_rng(seed=100),
        error=10**-5,
        steep=True,
    )
    for n in [100, 300, 500]
]
data_steep = pd.concat(data_steep)

In [ ]:
# add an extra column to distinguish the source (i.e. type of hillclimb)
full_data = []
for d, is_steep in [(data, False), (data_steep, True)]:
    d = d.copy()
    d.insert(0, "steep", is_steep)
    full_data.append(d)
full_data = pd.concat(full_data)

We modify the above plotter a bit as we now want to group the data by column "steep" (in the same plot).

In [ ]:
class FullPlotter(Plotter):
    def __init__(self):
        self.colors["steep = False"] = "blue"
        self.colors["steep = True"] = "red"

    def get_data_group_column_names(self):
        return ["steep"]


FullPlotter().plot(full_data, quantum_factor=2, y_lower_lim=10)